In [2]:
import pandas as pd
import numpy as np

In [40]:
df = pd.read_csv("/mnt/L-HDD/try_dataset/splitted.csv", chunksize = 1000000)

In [41]:
mfcc_cols = [f"mfcc_{i}" for i in range(1,21)]
total_sum = np.zeros(len(mfcc_cols))
total_sq_sum = np.zeros(len(mfcc_cols))
total_count = 0

for chunk in df:
    mfcc_data = chunk[:2][mfcc_cols].values
    total_sum += mfcc_data.sum(axis=0)
    total_sq_sum += (mfcc_data ** 2).sum(axis=0)
    total_count += len(mfcc_data)
    display(chunk[:2][mfcc_cols])
    
mean = total_sum / total_count
var = (total_sq_sum / total_count) - (mean ** 2)
std = np.sqrt(var + 1e-8)



,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20
0,-80.074165,61.71707,0.382712,28.728064,2.686816,5.856698,-1.155729,9.878029,-9.466604,-4.693692,-14.788804,0.759417,-2.715230,-5.323258,-7.693117,-3.169186,-16.774567,-14.542158,-13.916822,-8.063294
1,-22.963512,66.81962,-14.395244,18.222395,8.481088,-1.553265,-9.099847,7.828202,-6.180804,-6.670016,-11.087318,2.981239,-9.924412,-13.790710,-6.698185,-4.795344,-17.032550,-12.905769,-12.657845,-6.734515


[-51.5188385   64.268345    -7.00626594  23.4752295    5.5839521
   2.15171625  -5.1277878    8.8531156   -7.82370415  -5.681854
 -12.938061     1.87032812  -6.3198211   -9.55698395  -7.1956512
  -3.982265   -16.9035585  -13.7239635  -13.2873335   -7.39890435]


In [112]:
for a in df:
    print(a["mfcc"].max())

In [51]:
path_x_columns = [
    col
    for col in df.columns
    if col.startswith("path_") and int(col.split("_")[1]) % 2 != 0
]
path_y_columns = [
    col
    for col in df.columns
    if col.startswith("path_") and int(col.split("_")[1]) % 2 == 0
]
a = df.fillna(0)

# display(a[a[path_x_columns].ge(512).any(axis=1)][path_x_columns])
# display(a[a[path_y_columns].ge(385).any(axis=1)][path_y_columns])
display(a[a[path_y_columns].lt(0).any(axis=1)][path_y_columns])
display(a[a[path_x_columns].lt(0).any(axis=1)][path_x_columns])

,path_2,path_4,path_6,path_8,path_10,path_12,path_14,path_16,path_18,path_20,...,path_28,path_30,path_32,path_34,path_36,path_38,path_40,path_42,path_44,path_46


,path_1,path_3,path_5,path_7,path_9,path_11,path_13,path_15,path_17,path_19,...,path_27,path_29,path_31,path_33,path_35,path_37,path_39,path_41,path_43,path_45


In [60]:
a["rms"].max()

np.float64(0.70594484)

In [5]:
values = {"type": "slient", "new_combo": False}

df.fillna(value=values, inplace=True)

/tmp/ipykernel_35981/595422252.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(value=values, inplace=True)


In [6]:
df.columns[df.isna().any(axis=0)]

Index(['x', 'y', 'hit_sound', 'repeat', 'length', 'spinner_time',
       'beat_length', 'meter', 'slider_velocity', 'sample_set', 'volume',
       'effects', 'difficulty_rating', 'path_1', 'path_2', 'path_3', 'path_4',
       'path_5', 'path_6', 'path_7', 'path_8', 'path_9', 'path_10', 'path_11',
       'path_12', 'path_13', 'path_14', 'path_15', 'path_16', 'path_17',
       'path_18', 'path_19', 'path_20', 'path_21', 'path_22', 'path_23',
       'path_24', 'path_25', 'path_26', 'path_27', 'path_28', 'path_29',
       'path_30', 'path_31', 'path_32', 'path_33', 'path_34', 'path_35',
       'path_36', 'path_37', 'path_38', 'path_39', 'path_40', 'path_41',
       'path_42', 'path_43', 'path_44', 'path_45', 'path_46'],
      dtype='object')

In [7]:
fill_map = df[df["difficulty_rating"].notna()].groupby("id")["difficulty_rating"].first()
fill_map

id
2337169-0    4.42
2337726-0    4.49
2338303-0    4.25
2340739-0    3.53
2340739-1    2.34
2340739-2    4.67
2340739-3    5.65
2342033-0    5.38
2342033-1    4.89
2342033-2    4.72
2342033-3    3.57
382400-0     6.42
Name: difficulty_rating, dtype: float64

In [8]:
df["difficulty_rating"] = df["id"].map(fill_map)

In [9]:
df.columns[df.isna().any(axis=0)]

Index(['x', 'y', 'hit_sound', 'repeat', 'length', 'spinner_time',
       'beat_length', 'meter', 'slider_velocity', 'sample_set', 'volume',
       'effects', 'path_1', 'path_2', 'path_3', 'path_4', 'path_5', 'path_6',
       'path_7', 'path_8', 'path_9', 'path_10', 'path_11', 'path_12',
       'path_13', 'path_14', 'path_15', 'path_16', 'path_17', 'path_18',
       'path_19', 'path_20', 'path_21', 'path_22', 'path_23', 'path_24',
       'path_25', 'path_26', 'path_27', 'path_28', 'path_29', 'path_30',
       'path_31', 'path_32', 'path_33', 'path_34', 'path_35', 'path_36',
       'path_37', 'path_38', 'path_39', 'path_40', 'path_41', 'path_42',
       'path_43', 'path_44', 'path_45', 'path_46'],
      dtype='object')

In [10]:
df.fillna(0, inplace=True)

In [11]:
df.columns[df.isna().any(axis=0)]

Index([], dtype='object')

In [12]:
df

,id,time,type,x,y,hit_sound,repeat,length,spinner_time,new_combo,...,path_37,path_38,path_39,path_40,path_41,path_42,path_43,path_44,path_45,path_46
0,2338303-0,1375.000000,circle,101.0,325.0,0.0,0.0,0.00,0.0,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2338303-0,1381.587302,slient,0.0,0.0,0.0,0.0,0.00,0.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2338303-0,1393.197279,slient,0.0,0.0,0.0,0.0,0.00,0.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2338303-0,1404.807256,slient,0.0,0.0,0.0,0.0,0.00,0.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2338303-0,1416.417234,slient,0.0,0.0,0.0,0.0,0.00,0.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234295,2342033-3,221448.707483,slient,0.0,0.0,0.0,0.0,0.00,0.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
234296,2342033-3,221460.317460,slient,0.0,0.0,0.0,0.0,0.00,0.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
234297,2342033-3,221471.927438,slient,0.0,0.0,0.0,0.0,0.00,0.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
234298,2342033-3,221483.537415,slient,0.0,0.0,0.0,0.0,0.00,0.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df.columns

Index(['id', 'time', 'type', 'x', 'y', 'hit_sound', 'repeat', 'length',
       'spinner_time', 'new_combo', 'beatmap_id', 'beat_length', 'meter',
       'slider_velocity', 'sample_set', 'volume', 'effects',
       'difficulty_rating', 'frame_time', 'rms', 'has_hit_object', 'mfcc_1',
       'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8',
       'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 'mfcc_14',
       'mfcc_15', 'mfcc_16', 'mfcc_17', 'mfcc_18', 'mfcc_19', 'mfcc_20',
       'curve_type', 'path_1', 'path_2', 'path_3', 'path_4', 'path_5',
       'path_6', 'path_7', 'path_8', 'path_9', 'path_10', 'path_11', 'path_12',
       'path_13', 'path_14', 'path_15', 'path_16', 'path_17', 'path_18',
       'path_19', 'path_20', 'path_21', 'path_22', 'path_23', 'path_24',
       'path_25', 'path_26', 'path_27', 'path_28', 'path_29', 'path_30',
       'path_31', 'path_32', 'path_33', 'path_34', 'path_35', 'path_36',
       'path_37', 'path_38', 'path_39', 'path_40', 

In [1]:
import pandas as pd
df = pd.read_csv("/mnt/L-HDD/try_dataset/normalized.csv")

In [2]:
dict(df.dtypes)

{'id': dtype('O'),
 'time': dtype('float64'),
 'x': dtype('float64'),
 'y': dtype('float64'),
 'repeat': dtype('float64'),
 'length': dtype('float64'),
 'spinner_time': dtype('float64'),
 'beatmap_id': dtype('int64'),
 'beat_length': dtype('float64'),
 'meter': dtype('float64'),
 'slider_velocity': dtype('float64'),
 'volume': dtype('float64'),
 'frame_time': dtype('float64'),
 'rms': dtype('float64'),
 'mfcc_1': dtype('float64'),
 'mfcc_2': dtype('float64'),
 'mfcc_3': dtype('float64'),
 'mfcc_4': dtype('float64'),
 'mfcc_5': dtype('float64'),
 'mfcc_6': dtype('float64'),
 'mfcc_7': dtype('float64'),
 'mfcc_8': dtype('float64'),
 'mfcc_9': dtype('float64'),
 'mfcc_10': dtype('float64'),
 'mfcc_11': dtype('float64'),
 'mfcc_12': dtype('float64'),
 'mfcc_13': dtype('float64'),
 'mfcc_14': dtype('float64'),
 'mfcc_15': dtype('float64'),
 'mfcc_16': dtype('float64'),
 'mfcc_17': dtype('float64'),
 'mfcc_18': dtype('float64'),
 'mfcc_19': dtype('float64'),
 'mfcc_20': dtype('float64'),
 'p

In [3]:
df

,id,time,x,y,repeat,length,spinner_time,beatmap_id,beat_length,meter,...,new_combo_0,new_combo_1,difficulty_rating_0,difficulty_rating_1,difficulty_rating_7,difficulty_rating_8,difficulty_rating_9,difficulty_rating_10,difficulty_rating_11,difficulty_rating_12
0,2338303-0,1375.000000,0.197266,0.844156,0.000000,0.000000,0.0,2338303,6.034684,1.609438,...,False,False,False,False,False,False,False,False,False,False
1,2338303-0,1381.587302,0.000000,0.000000,0.000000,0.000000,0.0,2338303,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
2,2338303-0,1393.197279,0.000000,0.000000,0.000000,0.000000,0.0,2338303,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
3,2338303-0,1404.807256,0.000000,0.000000,0.000000,0.000000,0.0,2338303,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
4,2338303-0,1416.417234,0.000000,0.000000,0.000000,0.000000,0.0,2338303,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234295,2342033-3,221448.707483,0.000000,0.000000,0.000000,0.000000,0.0,2342033,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
234296,2342033-3,221460.317460,0.000000,0.000000,0.000000,0.000000,0.0,2342033,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
234297,2342033-3,221471.927438,0.000000,0.000000,0.000000,0.000000,0.0,2342033,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False
234298,2342033-3,221483.537415,0.000000,0.000000,0.000000,0.000000,0.0,2342033,0.000000,0.000000,...,False,False,False,False,False,False,False,False,False,False


In [1]:
import pandas as pd

In [12]:
df = pd.read_csv("/mnt/L-HDD/Downloads/merged.csv", chunksize=5000000)

In [13]:
ids = []
for chunk in df:
    ids = ids + list(chunk["beatmap_id"].unique())

In [14]:
setid = list(set(ids))

In [15]:
with open("/mnt/L-HDD/Downloads/processed.txt", "w") as f:
    for item in setid:
        f.write(f"{item}\n")